In [98]:
from __future__ import print_function 
import boto3
import os
import pandas as pd
from flask import Flask
from pandas.io.json import json_normalize
import numpy as np
from datetime import datetime
from dateutil.parser import parse
from statistics import mean 
from datetime import timedelta

In [39]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

In [40]:
table = dynamodb.Table("hungrymind-mobilehub-593518188-Book_Borrow")
data_downloaded2 = table.scan()

In [41]:
borrow = pd.DataFrame.from_dict(json_normalize(data_downloaded2['Items']))

In [43]:
borrow.drop(['CustId'],axis=1,inplace=True)
borrow.head()

,ActualRetDate,BookID,BorrowId,CustID,DateClaimToRet,DateOfBorrow,Rating,SupplierID
0,1-12-1924,786907509,4666,1440,27-11-1924,20-11-1924,5,1
1,24-8-1967,310276993,3389,1435,22-8-1967,2-8-1967,4,1
2,1-5-1988,452282829,7315,1272,30-4-1988,19-4-1988,5,1
3,28-8-2015,805445528,1838,1258,26-8-2015,17-8-2015,1,1
4,27-9-1967,1591840538,7455,1125,27-9-1967,28-8-1967,0,1


In [44]:
user_records = borrow.drop(['BorrowId','BookID','Rating','SupplierID'] , axis = 1)
user_records.head()

,ActualRetDate,CustID,DateClaimToRet,DateOfBorrow
0,1-12-1924,1440,27-11-1924,20-11-1924
1,24-8-1967,1435,22-8-1967,2-8-1967
2,1-5-1988,1272,30-4-1988,19-4-1988
3,28-8-2015,1258,26-8-2015,17-8-2015
4,27-9-1967,1125,27-9-1967,28-8-1967


In [45]:
book_records = borrow.drop(['BorrowId','CustID','Rating','SupplierID'] , axis = 1)
book_records.head()

,ActualRetDate,BookID,DateClaimToRet,DateOfBorrow
0,1-12-1924,786907509,27-11-1924,20-11-1924
1,24-8-1967,310276993,22-8-1967,2-8-1967
2,1-5-1988,452282829,30-4-1988,19-4-1988
3,28-8-2015,805445528,26-8-2015,17-8-2015
4,27-9-1967,1591840538,27-9-1967,28-8-1967


In [46]:
current_user = 1448

In [47]:
curr_user_records = user_records[user_records.CustID == current_user]
curr_user_records.head()

,ActualRetDate,CustID,DateClaimToRet,DateOfBorrow
253,2-10-1983,1448,3-10-1983,18-9-1983
291,12-10-1928,1448,11-10-1928,22-9-1928
374,23-11-1903,1448,19-11-1903,29-10-1903
856,NaN,1448,04-06-2019,25-5-2019
932,19-7-1974,1448,17-7-1974,27-6-1974


In [48]:
curr_user_records.drop(['CustID'] , axis = 1 ,inplace=True)
curr_user_records.head()

,ActualRetDate,DateClaimToRet,DateOfBorrow
253,2-10-1983,3-10-1983,18-9-1983
291,12-10-1928,11-10-1928,22-9-1928
374,23-11-1903,19-11-1903,29-10-1903
856,NaN,04-06-2019,25-5-2019
932,19-7-1974,17-7-1974,27-6-1974


In [53]:
current_book = 385333218

In [57]:
curr_book_records = book_records[book_records.BookID == current_book]
curr_book_records.head()

,ActualRetDate,BookID,DateClaimToRet,DateOfBorrow
856,NaN,385333218,04-06-2019,25-5-2019
2405,24-9-1998,385333218,25-9-1998,11-9-1998
3682,27-11-1947,385333218,20-11-1947,6-11-1947


In [58]:
curr_book_records.drop(['BookID'] , axis = 1 ,inplace=True)
curr_book_records.head()

,ActualRetDate,DateClaimToRet,DateOfBorrow
856,NaN,04-06-2019,25-5-2019
2405,24-9-1998,25-9-1998,11-9-1998
3682,27-11-1947,20-11-1947,6-11-1947


In [62]:
to_be_predicted = curr_book_records[curr_book_records['ActualRetDate'].isnull()]
to_be_predicted

,ActualRetDate,DateClaimToRet,DateOfBorrow
856,NaN,04-06-2019,25-5-2019


In [63]:
curr_book_records.dropna(inplace=True)
curr_book_records.head()

C:\Users\Lenovo\Anaconda3\envs\ishita\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ActualRetDate,DateClaimToRet,DateOfBorrow
2405,24-9-1998,25-9-1998,11-9-1998
3682,27-11-1947,20-11-1947,6-11-1947


In [64]:
curr_user_records.dropna(inplace=True)
curr_user_records.head()

C:\Users\Lenovo\Anaconda3\envs\ishita\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ActualRetDate,DateClaimToRet,DateOfBorrow
253,2-10-1983,3-10-1983,18-9-1983
291,12-10-1928,11-10-1928,22-9-1928
374,23-11-1903,19-11-1903,29-10-1903
932,19-7-1974,17-7-1974,27-6-1974
1071,24-10-1949,17-10-1949,22-9-1949


In [106]:
date = to_be_predicted.DateClaimToRet.values
datestr = str(date[0])
curr_date_claim = datetime.strptime(datestr, '%d-%m-%Y')
curr_date_claim 

datetime.datetime(2019, 6, 4, 0, 0)

In [52]:
def find_average(records):
    actual_date = []
    claimed_date = []

    for i in range(len(records)):
        actual_date.append(records.iloc[i,:].ActualRetDate)
        claimed_date.append(records.iloc[i,:].DateClaimToRet)
        
    difference = []
    for i in range(len(actual_date)):
        ad = datetime.strptime(str(actual_date[i]), '%d-%m-%Y')
        cd = datetime.strptime(str(claimed_date[i]), '%d-%m-%Y')
        difference.append(int((cd-ad).days))
     
    average = mean(difference)  
    return average

In [29]:
average_cust = find_average(curr_user_records)
average_book = find_average(curr_book_records)
print(average_cust,average_book)

-2.0833333333333335 -3.5


In [30]:
total_avg = average_cust+average_book
total_avg = total_avg/2

In [105]:
round(total_avg)

-3

In [109]:
predicted_date = curr_date_claim - timedelta(days=round(total_avg))
predicted_date_str = str(predicted_date.day)+"-"+str(predicted_date.month)+"-"+str(predicted_date.year)
predicted_date_str

'7-6-2019'